# Installing Semantic Kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha"

#!import Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha

In [2]:
#pragma warning disable SKEXP0050
var bingConnector = new BingConnector("d1dfed6c3bd24b5596becde4bec6d35c");
var bing = new WebSearchEnginePlugin(bingConnector);
#pragma warning restore SKEXP0050

kernel.ImportPluginFromObject(bing, "bing");

### 📋 Add some arguments

In [3]:
var promptTemplate = "When is the next total {{$input}} eclipse?";
var function = kernel.Plugins["bing"]["search"];
var bingResult = await kernel.InvokeAsync(function, new() { ["query"] = promptTemplate });

var currentContext = bingResult.GetValue<string>();
display(currentContext)

["When is the next eclipse in your city? Eclipses in 2026. Aug 12Solar Eclipse (Total) Eclipses in 2027. Aug 2Solar Eclipse (Total) Eclipses in 2028. Jul 22Solar Eclipse (Total) Eclipses in 2030. Nov 25Solar Eclipse (Total) Eclipses in 2031. Nov 14Solar Eclipse (Total) Eclipses in 2033. Mar 30Solar Eclipse (Total) Eclipses in 2034.","It's been a long time since a total solar eclipse passed through Europe, but on Aug. 12, 2026, totality will come to the extreme northwest and southwest of the continent. Visible in Greenland ...","Explore a map of the next 15 total solar eclipses. In case you miss this year’s solar eclipse, there are 14 more in the next 20 years. This map of eclipse paths from 2024 to 2044...","According to NASA, after Monday's total solar eclipse, the next one viewable from the contiguous U.S. will be on Aug. 23, 2044. Total solar eclipse 2024: Watch video and see photos of the...","When is the next total solar eclipse? Total solar eclipses happen about every year or two

### 🏃‍♀️ Run prompt

In [4]:
var promptTemplate = """
    When is the next total {{$input}} eclipse?

    ### real-time search result context
    {{$currentContext}} 

    Respond only with JSON.
    """;

var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 500,
     Temperature = 0.1 
};
var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "solar",
    ["currentContext"] = currentContext
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

```json
{
  "next_total_solar_eclipse": {
    "date": "August 12, 2026",
    "visibility": {
      "regions": [
        "Greenland",
        "extreme northwest of Europe",
        "southwest of Europe"
      ],
      "note": "mostly over the Arctic Ocean"
    }
  }
}
```

### 👀 See rendered prompt

In [5]:
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplateRenderer = promptTemplateFactory.Create(new PromptTemplateConfig(promptTemplate));
var renderedPrompt = await promptTemplateRenderer.RenderAsync(kernel, kernelArguments);

display(renderedPrompt)

When is the next total solar eclipse?

### real-time search result context
["When is the next eclipse in your city? Eclipses in 2026. Aug 12Solar Eclipse (Total) Eclipses in 2027. Aug 2Solar Eclipse (Total) Eclipses in 2028. Jul 22Solar Eclipse (Total) Eclipses in 2030. Nov 25Solar Eclipse (Total) Eclipses in 2031. Nov 14Solar Eclipse (Total) Eclipses in 2033. Mar 30Solar Eclipse (Total) Eclipses in 2034.","It's been a long time since a total solar eclipse passed through Europe, but on Aug. 12, 2026, totality will come to the extreme northwest and southwest of the continent. Visible in Greenland ...","Explore a map of the next 15 total solar eclipses. In case you miss this year’s solar eclipse, there are 14 more in the next 20 years. This map of eclipse paths from 2024 to 2044...","According to NASA, after Monday's total solar eclipse, the next one viewable from the contiguous U.S. will be on Aug. 23, 2044. Total solar eclipse 2024: Watch video and see photos of the...","When is the ne